To be run from a local copy of SpriteSomething with the following minor modifications to get_tracker_images:

``` python
def get_tracker_images(self):
        '''
        Crossproduct Tracker Images
        '''
        return_images = []

        #cycle through mail levels
        for i,mail in enumerate(["green","blue","red"]):
            #get a container for tile lists
            tile_list = {}
            #get Bunny tile list for Stand:down to grab the bunny head
            tile_list["bunny"] = self.get_tiles_for_pose("Bunny stand","down",0,["bunny_mail" if mail == 'green' else mail + "_mail"],0)
            #get Link tile list for File select for base
            tile_list["link"] = self.get_tiles_for_pose("File select","right",0,[mail + "_mail"],0)
            #get the bunny head
            # bunny_head = tile_list["bunny"][2]
            #copy Link over Bunny
            # tile_list["bunny"] = tile_list["link"] + []
            #set the bunny head
            # tile_list["bunny"][1] = bunny_head

            #cycle through tile lists
            for tileset_id, tile_set in tile_list.items():
                #make src image from tile list
                src_img,_ = self.assemble_tiles_to_completed_image(tile_set)
                #crop out the actual pixels
                if tileset_id != "bunny":
                    src_img = src_img.crop((5,7,21,29))
                #make a new 32x32 transparent image
                dest_img = Image.new("RGBA",(32,32))
                #paste the pixels to (7,7)
                dest_img.paste(src_img,(7,6))
                #resize using nearest neighbor to 400% because that's what Cross' tracker uses
                dest_img = dest_img.resize((32*4,32*4),Image.NEAREST)
                #save to disk
                filename = "tunic"
                if tileset_id == "bunny":
                    filename += "bunny"
                filename += str(i+1)
                filename += ".png"
                return_images.append((filename,dest_img))

        return return_images
```

In [ ]:
from source.snes.zelda3.link import sprite as link_sprite
import ast
from tests.legacy.common_vars import LINK_RESOURCE_SUBPATH
import re
import requests

In [ ]:
baseurl = 'https://alttpr-assets.s3.us-east-2.amazonaws.com/'

sprite_list = requests.get(baseurl + 'sprites.json').json()
php_list = requests.get('https://github.com/sporchia/alttp_vt_randomizer/raw/master/config/sprites.php').text

In [ ]:
php_list = php_list.replace('\n', '').replace('<?php', '').replace(';', '').replace(' => ', ': ').replace('[', '{').replace(']', '}').replace('return', '')

In [ ]:
php_list = php_list[1:]

In [ ]:
all_sprites = ast.literal_eval(php_list)

In [ ]:
all_sprites_list = [all_sprites[sprite] for sprite in all_sprites]

In [ ]:
for sprite in sprite_list:
    if sprite['file'] not in all_sprites:
        print(f"Missing sprite {sprite['file']}")
        all_sprites_list.append(sprite)

In [ ]:
sprite_list_sorted = sorted(all_sprites_list, key=lambda x: x['name'].lower())

In [ ]:
import os

In [ ]:
for sprite in sprite_list_sorted:
    if os.path.exists(f"sprites/{sprite['file']}"):
        continue
    r = requests.get(baseurl + sprite['file'])
    if r.status_code != 200:
        print(f"Failed to download {sprite['file']}")
        continue
    with open(f"sprites/{sprite['file'].split('/')[-1]}", 'wb') as f:
        f.write(r.content)


In [ ]:
len(sprite_list_sorted)

In [ ]:
import collections as c

In [ ]:
processed_sprites = {}
to_delete = []
for n, sprite in enumerate(sprite_list_sorted):
    if sprite["name"] in processed_sprites and processed_sprites[sprite["name"]] > sprite["version"]:
        to_delete.append(n)
        continue
    processed_sprites[sprite["name"]] = sprite["version"]

In [ ]:
to_delete

In [ ]:
for n in reversed(to_delete):
    del sprite_list_sorted[n]

In [ ]:
with open('sprites/001.link.1.zspr', 'wb') as f:
    link = requests.get('https://github.com/Zarby89/Enemizer/raw/09b890d15560cb1c810832bdc97a440ae569c55e/EnemizerCLI.Core/sprites/001.link.1.zspr').content
    f.write(link)

In [ ]:
import json

In [ ]:
css_names = {
    "tunicbunny1.png": "active-1.bunny",
    "tunicbunny2.png": "active-2.bunny",
    "tunicbunny3.png": "active-3.bunny",
    "tunic1.png": "active-1",
    "tunic2.png": "active-2",
    "tunic3.png": "active-3",
}
manifest_dict = json.load(open('resources/app/snes/zelda3/manifests/manifest.json', 'r'))
with open('sprites.css', 'w') as fcss, open('sprite_selection.html', 'w') as fhtml:
    for sprite in sprite_list_sorted:
        manifest_dict['name'] = sprite['name']
        ss_sprite = link_sprite.Sprite(
            f'./sprites/{sprite["file"]}',
            manifest_dict=manifest_dict,
            my_subpath=LINK_RESOURCE_SUBPATH,
            sprite_name=sprite["name"],
        )
        images = ss_sprite.get_representative_images(style="crossproduct")
        sprite_pname = re.sub('[^0-9a-zA-Z]+', '_', sprite["name"])
        while "__" in sprite_pname:
            sprite_pname = sprite_pname.replace("__", "_")
        sprite_pname = sprite_pname.strip('_').strip().lower()
        if sprite_pname == 'tunic':
            sprite_pname = 'tunic_'
        images = sorted(images, key=lambda x: x[0])
        for i, (img_name, img) in enumerate(images):
            img.save(f"./sprite_images/{sprite_pname}_{img_name}")
            fcss.write(f'.tunic.{sprite_pname}.{css_names.get(img_name, img_name)}' + ' { ' + f'background-image: url(../images/sprites/{sprite_pname}_{img_name}); background-size:100%; float:left; ' + '}\n')
        if sprite["name"] == "Link":
            fhtml.write(f'<option value="{sprite_pname}" selected>{sprite["name"]}</option>\n')
        else:
            fhtml.write(f'<option value="{sprite_pname}">{sprite["name"]}</option>\n')


In [ ]:
# Manually add Link